<a href="https://colab.research.google.com/github/rlaalswls/Pneumonia_Detection_Project/blob/main/YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import os

print('🔧 환경 설정')
print('=' * 50)
print(f'PyTorch 버전: {torch.__version__}')
print(f'CUDA 사용 가능: {torch.cuda.is_available()}')

if torch.cuda.is_available():
    print(f'GPU 이름: {torch.cuda.get_device_name(0)}')
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'GPU 메모리: {gpu_mem:.2f} GB')
else:
    print('⚠️ GPU 사용 불가. 런타임 > 런타임 유형 변경에서 GPU 선택 필요')
print('=' * 50)

🔧 환경 설정
PyTorch 버전: 2.9.0+cu126
CUDA 사용 가능: True
GPU 이름: NVIDIA A100-SXM4-40GB
GPU 메모리: 42.47 GB


In [2]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt -q

Cloning into 'yolov5'...
remote: Enumerating objects: 17779, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 17779 (delta 0), reused 0 (delta 0), pack-reused 17772 (from 1)
Receiving objects: 100% (17779/17779), 16.92 MiB | 30.08 MiB/s, done.
Resolving deltas: 100% (12128/12128), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 15.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive'

Mounted at /content/drive


In [4]:
DATASET = {
    "root": os.path.join(drive_path, 'roboflow', 'pneumonia-detector'),
}

DATASET["yaml"] = os.path.join(DATASET["root"], "data.yaml")
DATASET["runs"] = os.path.join(DATASET["root"], "runs")

In [ ]:
# #CLAHE 적용 시작
# DATASET["clahe_processed"] = os.path.join(DATASET["root"], "clahe_processed")

In [ ]:
# # Step 12. CLAHE 전처리 함수
# import cv2
# import shutil
# from tqdm import tqdm

# def apply_clahe_to_dataset(source_root, dest_root):
#     print("🔄 CLAHE 전처리 시작...")

#     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
#     splits = ['train', 'valid', 'test']

#     for split in splits:
#         src_img_dir = os.path.join(source_root, split, 'images')
#         dst_img_dir = os.path.join(dest_root, split, 'images')
#         src_label_dir = os.path.join(source_root, split, 'labels')
#         dst_label_dir = os.path.join(dest_root, split, 'labels')

#         os.makedirs(dst_img_dir, exist_ok=True)
#         os.makedirs(dst_label_dir, exist_ok=True)

#         image_files = list(Path(src_img_dir).glob('*.jpg')) + \
#                      list(Path(src_img_dir).glob('*.png')) + \
#                      list(Path(src_img_dir).glob('*.jpeg'))

#         print(f"\n처리 중: {split} ({len(image_files)}개 이미지)")

#         for img_path in tqdm(image_files):
#             img = cv2.imread(str(img_path))
#             if img is None:
#                 continue

#             if len(img.shape) == 3:
#                 gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#             else:
#                 gray = img

#             clahe_img = clahe.apply(gray)
#             clahe_bgr = cv2.cvtColor(clahe_img, cv2.COLOR_GRAY2BGR)

#             dst_path = os.path.join(dst_img_dir, img_path.name)
#             cv2.imwrite(dst_path, clahe_bgr)

#             label_name = img_path.stem + '.txt'
#             src_label = os.path.join(src_label_dir, label_name)
#             dst_label = os.path.join(dst_label_dir, label_name)

#             if os.path.exists(src_label):
#                 shutil.copy2(src_label, dst_label)

#     print("\n✅ CLAHE 전처리 완료!")

# # Step 13. CLAHE 적용
# apply_clahe_to_dataset(DATASET["root"], DATASET["clahe_processed"])

# # Step 14. CLAHE data.yaml 생성
# if not os.path.exists(DATASET["clahe_processed"]):
#     os.makedirs(DATASET["clahe_processed"], exist_ok=True)

# data_yaml_clahe = {
#     "path": DATASET["clahe_processed"],
#     "train": "train/images",
#     "val": "valid/images",
#     "test": "test/images",
#     "nc": 1,
#     "names": ["Pneumonia"]
# }

# clahe_yaml_path = os.path.join(DATASET["clahe_processed"], "data.yaml")
# with open(clahe_yaml_path, "w") as f:
#     yaml.dump(data_yaml_clahe, f)

# print(f"✅ CLAHE data.yaml 생성: {clahe_yaml_path}")


In [ ]:
# import cv2
# from matplotlib import pyplot as plt

# # 예시: train 이미지 5개 확인
# sample_images = list(Path(DATASET["clahe_processed"], "train/images").glob("*.jpg"))[:5]

# for img_path in sample_images:
#     img = cv2.imread(str(img_path))
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     plt.imshow(img)
#     plt.title(img_path.name)
#     plt.axis('off')
#     plt.show()


In [ ]:
# # Step 15. CLAHE 학습
# !python train.py \
#     --img {IMG_SIZE} \
#     --batch {BATCH_SIZE} \
#     --epochs {EPOCHS} \
#     --data {clahe_yaml_path} \
#     --weights yolov5s.pt \
#     --project {PROJECT_DIR} \
#     --name pneumonia_clahe \
#     --cache \
#     --patience 20

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# from pathlib import Path
# import os
# import pandas as pd

# # CLAHE 모델 결과 경로
# results_path_clahe = os.path.join(PROJECT_DIR, "pneumonia_clahe")
# results_csv_clahe = os.path.join(results_path_clahe, "results.csv")

# if os.path.exists(results_csv_clahe):
#     df = pd.read_csv(results_csv_clahe)
#     df.columns = df.columns.str.strip()
#     last_epoch = df.iloc[-1]

#     precision_cols = [col for col in df.columns if 'precision' in col.lower()]
#     recall_cols = [col for col in df.columns if 'recall' in col.lower()]

#     if precision_cols and recall_cols:
#         precision = last_epoch[precision_cols[0]]
#         recall = last_epoch[recall_cols[0]]

#         valid_labels_dir = os.path.join(DATASET["clahe_processed"], "valid", "labels")
#         valid_labels = list(Path(valid_labels_dir).glob("*.txt"))

#         total_gt_boxes = sum(len(open(f).readlines()) for f in valid_labels)

#         TP = int(recall * total_gt_boxes)
#         FN = total_gt_boxes - TP
#         FP = int((TP / precision) - TP) if precision > 0 else 0
#         TN = 0  # object detection에서는 보통 TN 계산 안 함

#         # Confusion Matrix (이미지로만)
#         cm = np.array([[TP, FN],
#                        [FP, TN]])

#         plt.figure(figsize=(6,5))
#         sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
#                     xticklabels=['Pneumonia', 'Background'],
#                     yticklabels=['Pneumonia', 'Background'],
#                     cbar_kws={'label': 'Count'})
#         plt.ylabel('Predicted')
#         plt.xlabel('Actual')
#         plt.title('Confusion Matrix - CLAHE (Counts)')

#         # 이미지 저장
#         cm_save_path = os.path.join(results_path_clahe, "confusion_matrix_counts.png")
#         plt.savefig(cm_save_path, dpi=150, bbox_inches='tight')
#         plt.show()


# baseline code 실행 부분 #

In [5]:
import yaml

data_yaml = {
    "path": DATASET["root"],
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "nc": 1,
    "names": ["Pneumonia"]
}

with open(DATASET["yaml"], "w") as f:
    yaml.dump(data_yaml, f)

print(f"✅ data.yaml 생성 완료: {DATASET['yaml']}")

✅ data.yaml 생성 완료: /content/drive/MyDrive/roboflow/pneumonia-detector/data.yaml


In [6]:
#Experiment Config
IMG_SIZE = 640
BATCH_SIZE = 32
EPOCHS = 15
PROJECT_DIR = DATASET["runs"]

In [7]:
# YOLOv5 학습 (Baseline - 증강 없음)
!python train.py \
    --img {IMG_SIZE} \
    --batch {BATCH_SIZE} \
    --epochs {EPOCHS} \
    --data {DATASET["yaml"]} \
    --weights yolov5s.pt \
    --project {PROJECT_DIR} \
    --name pneumonia_baseline \
    --cache \
    --patience 20

train: Caching images (9.0GB ram):  67% 7897/11796 [1:27:16<43:05,  1.51it/s]
^C


In [ ]:
2# 학습된 모델 찾기
import glob as gb

weight_files = gb.glob(f"{PROJECT_DIR}/**/best.pt", recursive=True)
if weight_files:
    BEST_WEIGHTS = weight_files[0]
    print(f"✅ 모델 발견: {BEST_WEIGHTS}")
else:
    BEST_WEIGHTS = os.path.join(PROJECT_DIR, "pneumonia_baseline", "weights", "best.pt")
    print(f"⚠️ 기본 경로 사용: {BEST_WEIGHTS}")

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

results_path = os.path.join(PROJECT_DIR, "pneumonia_baseline")
results_csv = os.path.join(results_path, "results.csv")

if os.path.exists(results_csv):
    df = pd.read_csv(results_csv)
    df.columns = df.columns.str.strip()
    last_epoch = df.iloc[-1]

    precision_cols = [col for col in df.columns if 'precision' in col.lower()]
    recall_cols = [col for col in df.columns if 'recall' in col.lower()]

    if precision_cols and recall_cols:
        precision = last_epoch[precision_cols[0]]
        recall = last_epoch[recall_cols[0]]

        # Validation GT 박스 수
        valid_labels_dir = os.path.join(DATASET["root"], "valid", "labels")
        valid_labels = list(Path(valid_labels_dir).glob("*.txt"))
        total_gt_boxes = sum(len(open(f).readlines()) for f in valid_labels)

        # TP, FP, FN 계산
        TP = int(recall * total_gt_boxes)
        FN = total_gt_boxes - TP
        FP = int((TP / precision) - TP) if precision > 0 else 0
        TN = 0

        # Confusion Matrix 시각화
        cm = np.array([[TP, FN],
                       [FP, TN]])

        plt.figure(figsize=(6,5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Pneumonia', 'Background'],
                    yticklabels=['Pneumonia', 'Background'],
                    cbar_kws={'label':'Count'})
        plt.ylabel('Predicted')
        plt.xlabel('Actual')
        plt.title('Confusion Matrix (Counts)')
        plt.show()
